# CityBikes

Send a request to CityBikes for the city of your choice. 

In [32]:
import requests
import pandas as pd

# Fetch the list of all networks
response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

# Check if the request was successful (status code 200)
if response.status_code == 200:
    city_name = input("Enter name of city: ")

    filtered_networks = [network for network in data["networks"] if network["location"]["city"].lower() == city_name.lower()]

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
        chosen_network_data = chosen_network_response.json()

        bike_stations = chosen_network_data.get("network", {}).get("stations", [])

        if bike_stations:
            # Create a DataFrame from the bike_stations list
            df = pd.DataFrame(bike_stations)

            print(f"Bike Stations in {city_name}:\n")
            print(df)
        else:
            print(f"No bike stations found for {city_name}.")
    else:
        print(f"No networks found for {city_name}.")
else:
    print(f"Failed to retrieve network data. Status code: {response.status_code}")


Bike Stations in madrid:

     empty_slots                                              extra  \
0             17  {'address': 'Calle Miguel Moya nº 1,', 'light'...   
1              9  {'address': 'Plaza del Conde del Valle de Súch...   
2             17  {'address': 'Calle Hortaleza nº 63,', 'light':...   
3             19  {'address': 'Calle Hortaleza nº 75,', 'light':...   
4             17  {'address': 'Plaza de San Miguel nº 9,', 'ligh...   
..           ...                                                ...   
389           12  {'address': 'Arturo Soria, 329,', 'light': 'ye...   
390           11  {'address': 'Calle del tejo,', 'light': 'yello...   
391           15  {'address': ' Calle Babilonia, 19,', 'light': ...   
392           16  {'address': 'Calle Albelda, 6,', 'light': 'yel...   
393           17  {'address': '311 - Calle del Profesor Arangure...   

     free_bikes                                id   latitude  longitude  \
0            10  82e78d26829a2dc89122e8f0988e2

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  394 non-null    int64  
 1   extra        394 non-null    object 
 2   free_bikes   394 non-null    int64  
 3   id           394 non-null    object 
 4   latitude     394 non-null    float64
 5   longitude    394 non-null    float64
 6   name         394 non-null    object 
 7   timestamp    394 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 24.8+ KB


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [34]:
# create a function to retrieve the columns we need
import requests
import pandas as pd

def get_bike_stations_data(city_name):
    filtered_networks = []
    for network in data["networks"]:
        if network["location"]["city"].lower() == city_name.lower():
            filtered_networks.append(network)

    if filtered_networks:
        chosen_network = filtered_networks[0]
        chosen_network_id = chosen_network["id"]

        chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
        chosen_network_data = chosen_network_response.json()

        bike_stations = chosen_network_data["network"]["stations"]

        # Create a list of dictionaries to store station data
        station_data_list = []

        for station in bike_stations:
            latitude = station["latitude"]
            longitude = station["longitude"]
            station_name = station["name"]  # Include station name

            # Check if 'ebikes' is in the 'extra' dictionary
            if 'extra' in station and 'ebikes' in station['extra']:
                total_bikes = station["free_bikes"] + station["empty_slots"] + station['extra']['ebikes']
            else:
                total_bikes = station["free_bikes"] + station["empty_slots"]


            station_data = {
                "Station Name": station_name,
                "Latitude": latitude,
                "Longitude": longitude,
                "Total Bikes": total_bikes
            }

            station_data_list.append(station_data)

        # Create a DataFrame from the list of dictionaries
        df = pd.DataFrame(station_data_list)

        print(f"Bike Stations in {city_name} (DataFrame):\n")
        print(df)
    else:
        print(f"No networks found for {city_name}.")

# Fetch the list of all networks
response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

# Take user input for the city name
city_name = city_name

# Call the function to retrieve and display bike station data
get_bike_stations_data(city_name)


Bike Stations in madrid (DataFrame):

                              Station Name   Latitude  Longitude  Total Bikes
0                          2 - Miguel Moya  40.420589  -3.705842           27
1                   3 - Plaza Conde Suchil  40.430294  -3.706917           19
2                  6 - Colegio Arquitectos  40.424148  -3.698447           19
3                            7 - Hortaleza  40.425191  -3.697771           19
4                  9 - Plaza de San Miguel  40.415606  -3.709508           24
..                                     ...        ...        ...          ...
389                494 - Arturo Soria, 330  40.480386  -3.667248           23
390                   272 - Calle del tejo  40.398001  -3.678027           23
391              559 - Calle Babilonia, 19  40.467530  -3.588235           23
392                       528 - Albelda, 6  40.449422  -3.657367           24
393  311 - Calle del Profesor Aranguren, 1  40.448931  -3.730518           27

[394 rows x 4 columns]


Put your parsed results into a DataFrame.

In [35]:
df.head(2)

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp
0,17,"{'address': 'Calle Miguel Moya nº 1,', 'light'...",10,82e78d26829a2dc89122e8f0988e2a28,40.420589,-3.705842,2 - Miguel Moya,2023-09-05T01:44:12.473000Z
1,9,{'address': 'Plaza del Conde del Valle de Súch...,10,248a049595da65f6aada65476ca1f958,40.430294,-3.706917,3 - Plaza Conde Suchil,2023-09-05T01:44:12.638000Z


In [36]:
# Already put in a dataframe in the function above.
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  394 non-null    int64  
 1   extra        394 non-null    object 
 2   free_bikes   394 non-null    int64  
 3   id           394 non-null    object 
 4   latitude     394 non-null    float64
 5   longitude    394 non-null    float64
 6   name         394 non-null    object 
 7   timestamp    394 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 24.8+ KB


In [38]:
df.to_csv(r'C:\Users\affuy\Documents\Data_Sets\df_2.csv')
